In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col log_pop, set log() for population data

In [2]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6 = "
  data {
    int N;
    int T[N];
    int N_societies;
    int society[N];
    int P[N];
  }
  parameters {
    real alpha;
    vector[N_societies] a_society;
    real bp;
    real<lower=0> sigma_society;
  }
  model {
    vector[N] mu;
    target += normal_lpdf(alpha | 0, 10);
    target += normal_lpdf(bp | 0, 1);
    target += cauchy_lpdf(sigma_society | 0, 1);
    target += normal_lpdf(a_society | 0, sigma_society);
    for(i in 1:N) mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
    target += poisson_log_lpmf(T | mu);
  }
  generated quantities {
    vector[N] log_lik;
    {
    vector[N] mu;
    for(i in 1:N) {
      mu[i] = alpha + a_society[society[i]] + bp * log(P[i]);
      log_lik[i] = poisson_log_lpmf(T[i] | mu[i]);
    }
    }
  }
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6",  model=m12_6, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_data = Dict("N" => size(d, 1), "T" => d[:total_tools], "N_societies" => 10, "society" => d[:society], "P" => d[:population]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.stan' at line 19)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.



Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__ -39.240903800  3.8779156097 0.061315229503 0.3335415827  135.175112
accept_stat__   0.936998380  0.1021632469 0.001615342767 0.0059526983  294.551763
   stepsize__   0.052680850  0.0042965123 0.000067933824 0.0006879066   39.009752
  treedepth__   5.809750000  0.5492088356 0.008683754158 0.0422818496  168.720198
 n_leapfrog__  69.266500000 31.5581519895 0.498978195162 1.4489128351  474.393367
  divergent__   0.000000000  0.0000000000 0.000000000000 0.0000000000         NaN
     energy__  45.730022600  4.5721042604 0.072291315814 0.3333296799  188.141360
        alpha   1.153559702  0.7082778814 0.011198856607 0.0238940445  878.674502
  a_society.1  -0.209504721  0.2422121942 0.003829711054 0.0074788130 1000.000000
  a_society.2   0.034049864  0.2130972863 0.003369363940 0.00558

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*